In [ ]:
# Load the extension
%load_ext sql

In [ ]:
# Connect to DuckDB
# NOTE: ONLY 1 NOTEBOOK CAN BE CONNECTED TO DUCKDB AT ANY TIME
%sql duckdb:///../../tpch.db

## [Exercise] 

Write a query to remove duplicates from the clickstream data (created as CTE below)

**Time limit during live workshop: 5 min**

**Hint:**
  1. Think about how you can use `row_number` as shown above to remove duplicates.

In [ ]:
%%sql
WITH clickstream AS (
    SELECT
        1 AS user_id, '2024-07-01 10:00:00' AS click_time UNION ALL
    SELECT
        1 AS user_id, '2024-07-01 10:05:00' AS click_time UNION ALL
    SELECT
        1 AS user_id, '2024-07-01 10:10:00' AS click_time UNION ALL
    SELECT
        1 AS user_id, '2024-07-01 10:10:00' AS click_time UNION ALL
    SELECT
        1 AS user_id, '2024-07-01 10:10:00' AS click_time UNION ALL
    SELECT
        1 AS user_id, '2024-07-01 10:10:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:15:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:20:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:20:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:20:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:20:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:25:00' AS click_time
),
ranked_clicks AS (
    SELECT
        user_id,
        click_time,
        ROW_NUMBER() OVER (PARTITION BY user_id, click_time) AS click_rank
    FROM
        clickstream
)
SELECT
    user_id,
    click_time,
    click_rank
FROM
    ranked_clicks
WHERE
    click_rank = 1;


## [Exercise] 

Write a query to pivot orders data by orderpriority and show average total price grouped by year
    
**Time limit during live workshop: 5 min**

**Hint**: 
    1. Use `strftime(o_orderdate, '%Y') AS order_year` to get order_year.

`orders` table schema: ![Orders](../../images/orders.png)

In [ ]:
%%sql
SELECT strftime(o_orderdate, '%Y') AS order_year,
       ROUND(AVG(CASE
                     WHEN o_orderpriority = '1-URGENT' THEN o_totalprice
                     ELSE NULL
                 END), 2) AS urgent_order_avg_price,
       ROUND(AVG(CASE
                     WHEN o_orderpriority = '2-HIGH' THEN o_totalprice
                     ELSE NULL
                 END), 2) AS high_order_avg_price,
       ROUND(AVG(CASE
                     WHEN o_orderpriority = '3-MEDIUM' THEN o_totalprice
                     ELSE NULL
                 END), 2) AS medium_order_avg_price,
       ROUND(AVG(CASE
                     WHEN o_orderpriority = '4-NOT SPECIFIED' THEN o_totalprice
                     ELSE NULL
                 END), 2) AS not_specified_order_avg_price,
       ROUND(AVG(CASE
                     WHEN o_orderpriority = '5-LOW' THEN o_totalprice
                     ELSE NULL
                 END), 2) AS low_order_avg_price
FROM orders
GROUP BY strftime(o_orderdate, '%Y');


## [Exercise]

* Scenario: You are designing a data set for a dashboard. The dashboard should be able to show metrics at day, week, month, and year levels (assume these are drop-downs on the dashboard).

* Assume that you, the data engineer assigned to building the table necessary for the dashboard.

* Question 1: What clarifying questions would you ask the dashboard team?

* Question 2: How would you design the table to be used by the dashboard software? What are the considerations you need to be mindful of?



### Question 1: What clarifying questions would you ask the dashboard team?

1. **Metrics Scope:**
   - What specific metrics are required to be displayed on the dashboard (e.g., sales, revenue, number of orders, user sign-ups, etc.)?

2. **Filtering and Segmentation:**
   - Will the dashboard require filtering or segmentation based on dimensions like region, product category, user demographics, etc.?
   - Are there any drill-down capabilities (e.g., from year to quarter to month) that need to be supported?

3. **Performance Requirements:**
   - What are the performance expectations for the dashboard (e.g., should the data load in real-time, or is some delay acceptable)?
   - How often will the data be refreshed (real-time, hourly, daily, etc.)?

4. **Historical Data:**
   - How much historical data needs to be maintained and made available on the dashboard?

5. **Visualization Requirements:**
   - Are there any specific visualization requirements or preferences that might affect how the data is structured (e.g., time series charts, heatmaps, etc.)?

6. **Data Volume and Scalability:**
   - What is the expected data volume, and do you foresee this data volume growing significantly over time?
   - Should the design account for scalability to handle increasing data volume or additional metrics in the future?

### Question 2: How would you design the table to be used by the dashboard software? What are the considerations you need to be mindful of?

#### Table Design:

1. **Fact Table Structure:**
   - **Granularity:** Design a fact table at the finest level of granularity required, such as daily transactions or events. This allows for flexible aggregation at higher levels (weekly, monthly, yearly) as needed.
   - **Date Dimension:** Include a `date_key` foreign key that links to a date dimension table. This date dimension should include columns for the day, week, month, quarter, and year to facilitate easy aggregation.
   - **Metrics Columns:** Include columns for each metric required by the dashboard, such as `total_sales`, `total_orders`, `total_revenue`, etc.

2. **Date Dimension Table:**
   - **Date Hierarchy:** Design the date dimension table to include columns like `date`, `day_of_week`, `week_of_year`, `month`, `quarter`, and `year`.
   - **Fiscal Calendar:** If the organization uses a fiscal calendar, include fiscal year, fiscal quarter, and fiscal month columns.
   - **Special Dates:** Include flags or indicators for holidays, weekends, or other significant dates that might affect metrics.

3. **Pre-Aggregation and Summary Tables:**
   - **Aggregated Tables:** Consider creating pre-aggregated summary tables at the weekly, monthly, and yearly levels to improve query performance on the dashboard. These tables can be refreshed periodically.
   - **Partitioning:** Partition the data by date, week, or month to optimize query performance and manage large data volumes efficiently.

4. **Partitions and Optimization:**
   - **Partitions:** Create appropriate partitions on commonly queried columns, such as `date_key`, `order_id`, and `customer_id`, to improve query performance.
   - **Materialized Views:** If the database supports materialized views, consider using them to store pre-computed aggregates that are frequently accessed.

5. **Handling Historical Data:**
   - **Data Retention:** Implement a data retention strategy that balances performance and storage costs. For example, keep detailed daily data for the most recent years and aggregate older data.
   - **Archiving:** Archive historical data that is no longer required for the dashboard but may need to be retained for compliance or historical analysis.

6. **Data Quality and Validation:**
   - **Data Integrity:** Ensure referential integrity between the fact and dimension tables.
   - **Validation Checks:** Implement data validation checks to ensure accuracy, such as verifying that totals match expected values and that all dates are populated in the date dimension.

#### Considerations:

- **Performance:** The design should ensure that queries run quickly, even as data volume grows. Pre-aggregated tables, partitioning, and indexing are critical considerations.
- **Scalability:** The design should be scalable to handle increasing data volumes and the addition of new metrics or dimensions in the future.
- **Flexibility:** The table design should be flexible enough to support different levels of granularity and the ability to drill down from year to quarter to month to day.
- **Data Refresh:** Ensure that the data refresh process is efficient and that the dashboard displays up-to-date information based on the refresh frequency.
- **User Experience:** Design the data structure to support a smooth and responsive user experience on the dashboard, with minimal delays when switching between different views (day, week, month, year).